<a href="https://colab.research.google.com/github/Blazindev/C214/blob/main/DesafioTecnico_GabrielDiniz_Dhauz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
pip install mysql-connector-python

In [ ]:
pip install pandas

In [88]:
from decimal import Decimal

SyntaxError: invalid syntax (<ipython-input-88-a15486c16467>, line 1)

In [79]:
import mysql.connector
import pandas as pd


config = {
  'user': 'candidate_user',
  'password': 'D3@bGh664%$1VHv*',
  'host': 'dhauz-instance.cutloqirhpd7.us-east-1.rds.amazonaws.com',
  'database': 'db_hiring_test'
}

try:
    conn = mysql.connector.connect(**config)
    print("Conexão ao banco de dados MySQL bem-sucedida!")
except mysql.connector.Error as err:
    print(f"Erro ao conectar ao banco de dados: {err}")
    exit(1)

try:
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM db_hiring_test.raw_transactions_table")
    transactions_table = cursor.fetchall()
    df_transactions_table = pd.DataFrame(transactions_table, columns=cursor.column_names)

    cursor.execute("SELECT * FROM db_hiring_test.raw_transactions_fee")
    transactions_fee = cursor.fetchall()
    df_transactions_fee = pd.DataFrame(transactions_fee, columns=cursor.column_names)


except mysql.connector.Error as err:
    print(f"Erro ao executar a consulta: {err}")

finally:
    cursor.close()
    conn.close()


Conexão ao banco de dados MySQL bem-sucedida!


In [82]:
#Fase 1 – Análise sobre as transações:

# 1. Qual é o endereço (carteira) com maior volume de transações enviadas?
endereco_maior_volume_enviado = df_transactions_table.groupby('AddressOrigin')['TotalSent'].sum()
df_transactions_table['TotalSent'] = pd.to_numeric(df_transactions_table['TotalSent'], errors='coerce')
df_transactions_table = df_transactions_table.dropna(subset=['TotalSent'])
endereco_maior_volume_enviado = endereco_maior_volume_enviado.idxmax()
print(endereco_maior_volume_enviado)

# 2. Qual é o dia do mês com maior volume de transações realizadas?
df_transactions_table['dia_do_mes'] = pd.to_datetime(df_transactions_table['SentDate'])
dia_maior_volume_transacoes = df_transactions_table.groupby('SentDate')['TotalSent'].sum()
dia_maior_volume_transacoes = dia_maior_volume_transacoes.idxmax()
print(dia_maior_volume_transacoes)

# 3. Em qual dia da semana geralmente mais transações são realizadas?
df_transactions_table['SentDate'] = pd.to_datetime(df_transactions_table['SentDate'])
df_transactions_table['dia_da_semana'] = df_transactions_table['SentDate'].dt.day_name()
grupo_dia_semana = df_transactions_table.groupby('dia_da_semana')['TotalSent'].sum()
dia_semana_maior_volume = grupo_dia_semana.idxmax()
print(dia_semana_maior_volume)

#4.Quais transações possuem condições atípicas e precisam ser validadas com o time responsável pela disponibilização dos dados?
# Calcular o primeiro e terceiro quartis
Q1 = df_transactions_table['TotalSent'].quantile(0.25)
Q3 = df_transactions_table['TotalSent'].quantile(0.75)
# Calcular IQR
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
transacoes_atipicas = df_transactions_table[(df_transactions_table['TotalSent'] < limite_inferior) | (df_transactions_table['TotalSent'] > limite_superior)]
if not transacoes_atipicas.empty:
    print("Transações atípicas:")
    print(transacoes_atipicas)
else:
    print("Não foram encontradas transações atípicas.")

#5.Qual a carteira com o maior saldo final? (considere que todas as carteiras estejam zeradas no início das análises e que seja possível existir saldo negativo).
saldo_final_por_carteira = df_transactions_table.groupby('AddressDestination')['TotalSent'].sum()
carteira_maior_saldo_final = saldo_final_por_carteira.idxmax()
print("A carteira com o maior saldo final é:", carteira_maior_saldo_final)


A-42
2021-01-23 16:05:49
Saturday
Não foram encontradas transações atípicas.
A carteira com o maior saldo final é: A-49


In [91]:
#Fase 2 – Análise sobre as taxas:

#1.Considerando que a carteira origem é responsável por pagar as taxas de envio, qual carteira seria responsável pelo maior pagamento de taxas em janeiro de 2021?
df_transactions_table['SentDate'] = pd.to_datetime(df_transactions_table['SentDate'])
transactions_jan_2021 = df_transactions_table[(df_transactions_table['SentDate'] >= '2021-01-01') & (df_transactions_table['SentDate'] <= '2021-01-31')]

fees_paid_by_wallet = transactions_jan_2021.groupby('AddressOrigin')['TotalSent'].sum() * df_transactions_fee['fee-percentage']
fees_paid_by_wallet = fees_paid_by_wallet.fillna(0)

wallet_with_highest_fees = fees_paid_by_wallet.idxmax()
print("Carteira responsável pelo maior pagamento de taxas em janeiro de 2021:", wallet_with_highest_fees)

#2.E em fevereiro de 2021?
df_transactions_table['SentDate'] = pd.to_datetime(df_transactions_table['SentDate'])

transactions_feb_2021 = df_transactions_table[(df_transactions_table['SentDate'] >= '2021-02-01') & (df_transactions_table['SentDate'] <= '2021-02-28')]

if transactions_feb_2021['TotalSent'].isnull().all():
    print("Todos os valores na coluna para o mês 'TotalSent' são NaN. Não é possível calcular o pagamento de taxas.")
else:
    fees_paid_by_wallet_feb = transactions_feb_2021.groupby('AddressOrigin')['TotalSent'].sum() * df_transactions_fee['fee-percentage']
    fees_paid_by_wallet_feb = fees_paid_by_wallet_feb.fillna(0)
    wallet_with_highest_fees = fees_paid_by_wallet.idxmax()
    print("Carteira responsável pelo maior pagamento de taxas em fevereiro de 2021:", wallet_with_highest_fees)

#3.Qual é o id da transação com a maior taxa paga?
def find_fee(total_sent):
    total_sent = Decimal(str(total_sent))
    for index, row in df_transactions_fee.iterrows():
        if total_sent >= row['range-start'] and total_sent <= row['range-end']:
            return float(row['fee-percentage'])
    return None

# Criando a nova coluna 'total_sent_fee' no DataFrame 'df_transactions_table'
df_transactions_table['total_sent_fee'] = df_transactions_table['TotalSent'].apply(lambda x: x * find_fee(x) if pd.notna(x) else None)
transaction_with_highest_fee_id = df_transactions_table.loc[df_transactions_table['total_sent_fee'].idxmax(), 'IdTransaction']
print("ID da transação com a maior taxa paga:", transaction_with_highest_fee_id)


#4 Calculando a média de taxa paga considerando todas as transações realizadas
mean_fee = df_transactions_table['total_sent_fee'].mean()
print("A média de taxa paga considerando todas as transações realizadas é:", mean_fee)

Carteira responsável pelo maior pagamento de taxas em janeiro de 2021: A-42
Todos os valores na coluna para o mês 'TotalSent' são NaN. Não é possível calcular o pagamento de taxas.
ID da transação com a maior taxa paga: ID1293
A média de taxa paga considerando todas as transações realizadas é: 6410.0


In [ ]:
#Fase 3 - Arquitetura:

'''
Para montar uma arquitetura na AWS para realizar as tarefas mencionadas (ingestão de dados por banco, garantia de qualidade dos dados, modelagem de dados, armazenamento e disponibilização dos dados), podemos utilizar:
O AWS Glue para realizar a ingestão de dados e para todo ETL e orquestração do pipeline. O Data Quality poderia ser feito utilizando serviços como AWS Glue DataBrew ou AWS Glue ETL Jobs.
Poderiamos usar Amazon Redshift para armazenar e modelar os dados. Para monitoramento, utilizaríamos o Amazon CloudWatch para monitorar métricas operacionais e configurar alertas sobre eventos importantes.

'''